This notebook create dataset config files from reference

In [28]:
import pandas as pd
import os
import glob
import json

In [29]:
os.getcwd()

'/neurospin/dico/jchavas/Runs/70_self-supervised_two-regions/Program/2023_jlaval_STSbabies/contrastive/notebooks/joel/utils'

In [30]:
# dataset = "UKB40"
# dataset_folder = "UkBioBank40"
dataset = "hcp"
dataset_folder = "hcp"

In [31]:
path = f"{os.getcwd()}/../../../configs/dataset/with_reskel_distbottom/2mm/{dataset}"
ref_file = f"{path}/reference.yaml"
crop_path = f"/neurospin/dico/data/deep_folding/current/datasets/{dataset_folder}/crops/2mm"
crop_dirs = glob.glob(f"{crop_path}/*")
crop_drops = ["deMatos.", "deMatos_polar.", "CENTRAL"]

In [32]:
crop_dirs

['/neurospin/dico/data/deep_folding/current/datasets/hcp/crops/2mm/F.Coll.-S.Rh.',
 '/neurospin/dico/data/deep_folding/current/datasets/hcp/crops/2mm/S.F.median-S.F.pol.tr.-S.F.sup.',
 '/neurospin/dico/data/deep_folding/current/datasets/hcp/crops/2mm/S.F.inf.-BROCA-S.Pe.C.inf.',
 '/neurospin/dico/data/deep_folding/current/datasets/hcp/crops/2mm/S.Po.C.',
 '/neurospin/dico/data/deep_folding/current/datasets/hcp/crops/2mm/fronto-parietal_medial_face.',
 '/neurospin/dico/data/deep_folding/current/datasets/hcp/crops/2mm/F.I.P.',
 '/neurospin/dico/data/deep_folding/current/datasets/hcp/crops/2mm/S.T.s.-S.GSM.',
 '/neurospin/dico/data/deep_folding/current/datasets/hcp/crops/2mm/CINGULATE.',
 '/neurospin/dico/data/deep_folding/current/datasets/hcp/crops/2mm/F.C.L.p.-S.GSM.',
 '/neurospin/dico/data/deep_folding/current/datasets/hcp/crops/2mm/S.C.-S.Po.C.',
 '/neurospin/dico/data/deep_folding/current/datasets/hcp/crops/2mm/CENTRAL',
 '/neurospin/dico/data/deep_folding/current/datasets/hcp/crops

In [33]:
crop_dirs = [f for f in crop_dirs if not os.path.basename(f) in crop_drops]

In [34]:
print(ref_file)
print('\n'.join(crop_dirs))

/neurospin/dico/jchavas/Runs/70_self-supervised_two-regions/Program/2023_jlaval_STSbabies/contrastive/notebooks/joel/utils/../../../configs/dataset/with_reskel_distbottom/2mm/hcp/reference.yaml
/neurospin/dico/data/deep_folding/current/datasets/hcp/crops/2mm/F.Coll.-S.Rh.
/neurospin/dico/data/deep_folding/current/datasets/hcp/crops/2mm/S.F.median-S.F.pol.tr.-S.F.sup.
/neurospin/dico/data/deep_folding/current/datasets/hcp/crops/2mm/S.F.inf.-BROCA-S.Pe.C.inf.
/neurospin/dico/data/deep_folding/current/datasets/hcp/crops/2mm/S.Po.C.
/neurospin/dico/data/deep_folding/current/datasets/hcp/crops/2mm/fronto-parietal_medial_face.
/neurospin/dico/data/deep_folding/current/datasets/hcp/crops/2mm/F.I.P.
/neurospin/dico/data/deep_folding/current/datasets/hcp/crops/2mm/S.T.s.-S.GSM.
/neurospin/dico/data/deep_folding/current/datasets/hcp/crops/2mm/CINGULATE.
/neurospin/dico/data/deep_folding/current/datasets/hcp/crops/2mm/F.C.L.p.-S.GSM.
/neurospin/dico/data/deep_folding/current/datasets/hcp/crops/2m

In [35]:
# Read in the reference file
with open(ref_file, 'r') as file:
  ref = file.read()

print(ref)

# @package dataset.REPLACE_DATASET
dataset_name: REPLACE_DATASET
pickle_normal: ${dataset_folder}/hcp/crops/2mm/REPLACE_CROP_NAME/mask/REPLACE_SIDEskeleton.pkl
numpy_all: ${dataset_folder}/hcp/crops/2mm/REPLACE_CROP_NAME/mask/REPLACE_SIDEskeleton.npy
subjects_all: ${dataset_folder}/hcp/crops/2mm/REPLACE_CROP_NAME/mask/REPLACE_SIDEskeleton_subject.csv
crop_dir: ${dataset_folder}/hcp/crops/2mm/REPLACE_CROP_NAME/mask/REPLACE_SIDEcrops
foldlabel_dir: ${dataset_folder}/hcp/crops/2mm/REPLACE_CROP_NAME/mask/REPLACE_SIDElabels
foldlabel_all: ${dataset_folder}/hcp/crops/2mm/REPLACE_CROP_NAME/mask/REPLACE_SIDElabel.npy
subjects_foldlabel_all: ${dataset_folder}/hcp/crops/2mm/REPLACE_CROP_NAME/mask/REPLACE_SIDElabel_subject.csv
distbottom_dir: ${dataset_folder}/hcp/crops/2mm/REPLACE_CROP_NAME/mask/REPLACE_SIDEdistbottom
distbottom_all: ${dataset_folder}/hcp/crops/2mm/REPLACE_CROP_NAME/mask/REPLACE_SIDEdistbottom.npy
subjects_distbottom_all: ${dataset_folder}/hcp/crops/2mm/REPLACE_CROP_NAME/mask/RE

In [36]:

def replace_reference_yaml(crop_dir, side, ref):
  """For each crop name, it builds the yaml from the reference yaml"""
  crop_name = os.path.basename(crop_dir)
  dataset_name = crop_name.replace('.', '')
  mask_file = f"{crop_dir}/mask/{side}mask_cropped.nii.gz.minf"
  print(mask_file)
  with open(mask_file, 'r') as file:
    mask = file.read()
  mask = mask.replace("attributes = ", "")
  mask = mask.replace("\'", "\"")
  # print(mask)
  mask_json = json.loads(mask)
  # print(mask_json)
  side = side
  side_long = "left" if side=='L' else "right"
  dataset_name = f"{dataset_name}_{side_long}"
  filedata = ref.replace('REPLACE_CROP_NAME', crop_name)
  filedata = filedata.replace('REPLACE_DATASET', dataset_name)
  filedata = filedata.replace('REPLACE_SIDE', side)
  filedata = filedata.replace('REPLACE_SIZEX', str(mask_json['sizeX']))
  filedata = filedata.replace('REPLACE_SIZEY', str(mask_json['sizeY']))
  filedata = filedata.replace('REPLACE_SIZEZ', str(mask_json['sizeZ']))

  result_file = f"{path}/{dataset_name}.yaml"

  return filedata, result_file

In [37]:
# Replace the target string
for crop_dir in crop_dirs:
    for side in ['L', 'R']:
        filedata, result_file = replace_reference_yaml(crop_dir, side, ref)
        print(result_file)
        with open(result_file, 'w') as file:
          file.write(filedata)
        

/neurospin/dico/data/deep_folding/current/datasets/hcp/crops/2mm/F.Coll.-S.Rh./mask/Lmask_cropped.nii.gz.minf
/neurospin/dico/jchavas/Runs/70_self-supervised_two-regions/Program/2023_jlaval_STSbabies/contrastive/notebooks/joel/utils/../../../configs/dataset/with_reskel_distbottom/2mm/hcp/FColl-SRh_left.yaml
/neurospin/dico/data/deep_folding/current/datasets/hcp/crops/2mm/F.Coll.-S.Rh./mask/Rmask_cropped.nii.gz.minf
/neurospin/dico/jchavas/Runs/70_self-supervised_two-regions/Program/2023_jlaval_STSbabies/contrastive/notebooks/joel/utils/../../../configs/dataset/with_reskel_distbottom/2mm/hcp/FColl-SRh_right.yaml
/neurospin/dico/data/deep_folding/current/datasets/hcp/crops/2mm/S.F.median-S.F.pol.tr.-S.F.sup./mask/Lmask_cropped.nii.gz.minf
/neurospin/dico/jchavas/Runs/70_self-supervised_two-regions/Program/2023_jlaval_STSbabies/contrastive/notebooks/joel/utils/../../../configs/dataset/with_reskel_distbottom/2mm/hcp/SFmedian-SFpoltr-SFsup_left.yaml
/neurospin/dico/data/deep_folding/current

In [38]:
# # Write the file out again
# with open('file.txt', 'w') as file:
#   file.write(filedata)

# Command line of train.py

In [39]:
path_champollion = "/neurospin/dico/data/deep_folding/current/models/Champollion_V0"
path_configs = path

In [40]:
trained_regions = glob.glob(f"{path_champollion}/*")
trained_regions = [f for f in trained_regions if not "README.md" in f]
trained_regions = [os.path.basename(f) for f in trained_regions]
print('\n'.join(trained_regions))

SC-sylv_left
SFinter-SFsup_left
STi-STs-STpol_right
FColl-SRh_right
STs-SGSM_left
SC-sylv_right
Lobule_parietal_sup_left
SFint-SR_left
SPoC_left
SFinf-BROCA-SPeCinf_right
SsP-SPaint_left
SOr-SOlf_left
SFinf-BROCA-SPeCinf_left
SFmedian-SFpoltr-SFsup_right
FColl-SRh_left
logs
STi-STs-STpol_left
SFint-FCMant_left
FCLp-subsc-FCLa-INSULA_right
OCCIPITAL_right
cluster
summary
SC-SPeC_right
SOr-SOlf_right
CINGULATE_left
SFint-SR_right
CINGULATE_right
FCLp-SGSM_right
fronto-parietal_medial_face_left
ScCal-SLi_left
FPO-SCu-ScCal_right
FPO-SCu-ScCal_left
OCCIPITAL_left
SC-SPeC_left
STs_right
FIP_left
STsbr_right
SC-SPoC_right
fronto-parietal_medial_face_right
SPeC_left
STs-SGSM_right
SPoC_right
STi-SOTlat_right
FCLp-SGSM_left
FCMpost-SpC_left
SFint-FCMant_right
SFmedian-SFpoltr-SFsup_left
SPeC_right
SFmarginal-SFinfant_left
FIP_right
FCLp-subsc-FCLa-INSULA_left
STsbr_left
SsP-SPaint_right
embeddings
ScCal-SLi_right
Lobule_parietal_sup_right
ORBITAL_left
UKB-RAP
SFmarginal-SFinfant_right
STi-SOTl

In [41]:
available_configs = glob.glob(f"{path_configs}/*")
available_configs = [f for f in available_configs if not "reference.yaml" in f]
available_configs = [os.path.basename(f).replace(".yaml", "") for f in available_configs]
print('\n'.join(available_configs))

SFint-FCMant_left
SOr-SOlf_left
CINGULATE_right
SFint-FCMant_right
SFinter-SFsup_left
SFmedian-SFpoltr-SFsup_left
SPoC_left
LARGE_CINGULATE_right
SOr-SOlf_right
SsP-SPaint_right
STi-SOTlat_left
ScCal-SLi_left
STsbr_left
SFint-SR_right
SsP-SPaint_left
STs_right
STs_left
SC-sylv_right
fronto-parietal_medial_face_left
FCLp-SGSM_right
SOr_right
ORBITAL_left
FIP_right
SC-SPeC_left
LARGE_CINGULATE_left
FCLp-subsc-FCLa-INSULA_right
Lobule_parietal_sup_left
SOr_left
FCMpost-SpC_right
SFinf-BROCA-SPeCinf_left
STsbr_right
SC-SPoC_left
FColl-SRh_left
SC-sylv_left
ORBITAL_right
SFinter-SFsup_right
STi-SOTlat_right
FColl-SRh_right
STi-STs-STpol_right
Lobule_parietal_sup_right
SC-SPoC_right
FCLp-SGSM_left
SC-SPeC_right
FIP_left
STi-STs-STpol_left
STs-SGSM_left
CENTRAL_left
SFmarginal-SFinfant_left
SPoC_right
SPeC_right
SFmarginal-SFinfant_right
SFinf-BROCA-SPeCinf_right
fronto-parietal_medial_face_right
SPeC_left
ScCal-SLi_right
CINGULATE_left
OCCIPITAL_left
FCLp-subsc-FCLa-INSULA_left
SFmedian-SFpo

In [42]:
to_train = set(available_configs) - set(trained_regions)
print(len(to_train))
to_train = [f"with_reskel_distbottom/2mm/{dataset}/{i}" for i in to_train]

5


In [43]:
','.join(to_train)

'with_reskel_distbottom/2mm/hcp/LARGE_CINGULATE_left,with_reskel_distbottom/2mm/hcp/SOr_right,with_reskel_distbottom/2mm/hcp/SOr_left,with_reskel_distbottom/2mm/hcp/LARGE_CINGULATE_right,with_reskel_distbottom/2mm/hcp/CENTRAL_left'

In [44]:
len(to_train)

5